In [222]:
import pandas as pd
import copy
import numpy as np
from collections import Counter 
import sklearn
from sklearn.preprocessing import PolynomialFeatures
import operator
from sklearn import linear_model
import scipy.io
from sklearn.model_selection import KFold



# run fmin on the loss function and gradient 

df1 = pd.read_csv("pressure.csv")
data1 = df1[['Houston']]
data1 = np.array(data1)

df2 = pd.read_csv("humidity.csv")
data2 = df2[['Houston']]
data2 = np.array(data2)

df3 = pd.read_csv("temperature.csv")
data3 = df3[['Houston']]
data3 = np.array(data3)

df4 = pd.read_csv("wind_speed.csv")
data4 = df4[['Houston']]
data4 = np.array(data4)

training = np.hstack(np.array([data1, data2, data3, data4]))


training = training[1:,:]

df5 = pd.read_csv("weather_description.csv")
data5 = df5[['Houston']]
data5 = np.array(data5)

#vector of weather classifications

output = data5[1:,:] #45252

print (training)

print (output)
#training = np.array(merge[:, :4])


#output = merge[:, 4:]





[[1.00900000e+03 9.30000000e+01 2.88270000e+02 1.00000000e+00]
 [1.00800000e+03 9.20000000e+01 2.88297576e+02 1.00000000e+00]
 [1.00800000e+03 9.00000000e+01 2.88334343e+02 1.00000000e+00]
 ...
 [1.02000000e+03 5.00000000e+01 2.96370000e+02 3.00000000e+00]
 [1.02000000e+03 4.90000000e+01 2.94650000e+02 2.00000000e+00]
 [1.02000000e+03 5.20000000e+01 2.91440000e+02 2.00000000e+00]]
[['sky is clear']
 ['sky is clear']
 ['sky is clear']
 ...
 ['sky is clear']
 ['haze']
 ['haze']]


In [223]:
list1 = []

for element in output:
	list1.append(str(element))

unique = list(set(list1))     #get list of unique weather classifications

dict1 = {}                    #map each weather classification to an integer in between 0 and 32 (i.e its index)
for i in range(len(unique)):
	dict1[unique[i]] = i

print (dict1)
dict2 = {}                    #map each integer from 0 to 32 to its weather classification (reverse of above for retreival later)
for element in dict1.keys():
    dict2[dict1[element]] = element

print (dict2)

rows = output.shape[0]
columns = output.shape[1]


for i in range(rows):         #convert each element in the training output vector to an integer from 0 to 32

	output[i,:] = dict1[str(output[i,:])]
    
print (output)

{"['sand']": 0, "['scattered clouds']": 1, "['thunderstorm']": 2, "['heavy intensity drizzle']": 3, "['thunderstorm with rain']": 4, "['broken clouds']": 5, "['mist']": 6, "['dust']": 7, "['sky is clear']": 8, "['overcast clouds']": 9, "['fog']": 10, "['proximity moderate rain']": 11, "['few clouds']": 12, "['shower rain']": 13, "['volcanic ash']": 14, "['light snow']": 15, "['thunderstorm with drizzle']": 16, "['squalls']": 17, "['light rain']": 18, "['proximity thunderstorm']": 19, "['proximity shower rain']": 20, "['proximity thunderstorm with rain']": 21, "['heavy intensity rain']": 22, "['thunderstorm with light drizzle']": 23, "['smoke']": 24, "['very heavy rain']": 25, "['thunderstorm with light rain']": 26, "['light intensity drizzle']": 27, "['heavy intensity shower rain']": 28, "['moderate rain']": 29, "['haze']": 30, "['thunderstorm with heavy rain']": 31, "['drizzle']": 32}
{0: "['sand']", 1: "['scattered clouds']", 2: "['thunderstorm']", 3: "['heavy intensity drizzle']", 4

In [224]:
X = np.nan_to_num(training) #convert all NaN to zero

rows2 = X.shape[0]
columns2 = X.shape[1]

average_of_columns = np.sum(X, axis = 0)/rows2 #average all columns 

for i in range(rows2):                  #each zero number becomes the average of its column 
    for j in range(columns2):
        if X[i,j] == 0:
            X[i,j] = average_of_columns[j]
    
    
p = 2
poly = sklearn.preprocessing.PolynomialFeatures(degree=p,include_bias=False)
X_poly = poly.fit_transform(X)
XX = np.vstack([np.ones((X_poly.shape[0],)),X_poly.T]).T           #add a column of ones to the beginning of the training matrix

#print (XX.shape)
    
y = np.nan_to_num(output)

y = np.ndarray.flatten(y)

average_of_y = np.sum(y)/len(y)

for i in range(len(y)):             #each zero number becomes the average of the vector

	if y[i] == np.nan:

		y[i] = int(average_of_y)
        
print(XX)
print(y)

[[1.00000000e+00 1.00900000e+03 9.30000000e+01 ... 8.30995929e+04
  2.88270000e+02 1.00000000e+00]
 [1.00000000e+00 1.00800000e+03 9.20000000e+01 ... 8.31154922e+04
  2.88297576e+02 1.00000000e+00]
 [1.00000000e+00 1.00800000e+03 9.00000000e+01 ... 8.31366936e+04
  2.88334343e+02 1.00000000e+00]
 ...
 [1.00000000e+00 1.02000000e+03 5.00000000e+01 ... 8.78351769e+04
  8.89110000e+02 9.00000000e+00]
 [1.00000000e+00 1.02000000e+03 4.90000000e+01 ... 8.68186225e+04
  5.89300000e+02 4.00000000e+00]
 [1.00000000e+00 1.02000000e+03 5.20000000e+01 ... 8.49372736e+04
  5.82880000e+02 4.00000000e+00]]
[8 8 8 ... 8 30 30]


In [226]:
def sigmoid (z):                      #sigmoid hypothesis function
    sig = 1/(1+np.exp(-1*z))
    return sig            
        
#print len(Counter(y))
def find_theta(X_train, X_test, y_train):                #find the vector of thetas (33 total)
    theta = []
    

  
    X_train = 1 + np.log(X_train)  #scale features
    
    """
    mu = np.mean(X,axis=0)
    sigma = np.std(X,axis=0)
    X = (X - mu) / sigma
    X[X == np.nan] = 1000
    """
    

    for i in range(len(unique)):             #the multi-class logistic regression will essentially run 33 binary logistic regresssions

        y1 = copy.deepcopy(y_train)

        for j in range(len(y_train)):     #when focusing on one of the 33 classifications, treat all others as 0 (binary)

            if y1[j] == i:

                y1[j] = 1

            else:

                 y1[j] = 0

        #print list(y1).count(1)

        reg =  100000                              #regularization term
        y1=y1.astype('int')
        from sklearn import linear_model
        sk_logreg_l2 = linear_model.LogisticRegression(C=1/reg,solver='lbfgs',fit_intercept=False)
        sk_logreg_l2.fit(X_train,y1)

        theta.append(np.ndarray.flatten(sk_logreg_l2.coef_))
        
    y_prob = []                                #you will have 33 probabilities from the 33 different hypothesis functions
    for element in theta:
        y_prob.append((sigmoid(X_test @ np.array(element))))
        #print (np.array(y_prob).shape)


    predictions = (np.argmax(y_prob,axis=0))       #for each example, find the theta that gives it the max probability
    #print (dict2)
   
    new = []
    for i in range(len(predictions)):
        new.append(dict2[predictions[i]])
            
    df5 = pd.read_csv("weather_description.csv")
    data5 = df5[['Houston']]
    data5 = np.array(data5)
    y_train = data5[1:,:]

    
    total = 0
    for i in range(len(new)):
    
        if str(y_train[i]) == new[i]: #compare the predicted output with the actual output
        
            total +=1
            
    return (float(total/(len(y_train))))


print ("Done")


Done


In [227]:
#print (final) 
print (find_theta(XX, XX, y))

0.2796561477945726
